# Titanic : Machine Learning from Disaster

# Part 1 - Data Preprocessing

## 1. Read Data

In [3]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

In [4]:
# Read data from files
data = pd.read_csv('data_set/train.csv')
print('------ Traing data ------')
print(data.info())

------ Traing data ------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None


## 2. Preprocessing
####  i. Fill in Null Data
####  ii. Change 'Sex' part to int （male = 1， female = 0）
####  iii. Keep the integer part, ignore char part

In [22]:
print('------------ Before Preprocessing ------------')
# print the data before preprocessing
dataset_X = data
# Type is DataFrame
print(type(dataset_X))

# print the first 2 line of data
print(dataset_X.iloc[0:2,:])

print('\n')
print('------------ Start Preprocessing ------------')
# fill in nan values with 0
data = data.fillna(0)

# convert ['male', 'female'] values to [1, 0]
data['Sex'] = data['Sex'].apply(lambda s: 1 if s == 'male' else 0)

# add 'Deceased' as another class
#data['Deceased'] = data['Survived'].apply(lambda s: 1 - s)

# select features and labels for training
dataset_X = data[['Sex', 'Age', 'Pclass', 'SibSp', 'Parch', 'Fare']]
#dataset_Y = data[['Deceased', 'Survived']]
dataset_Y = data[['Survived']]
print(dataset_X)
print(dataset_Y)

------------ Before Preprocessing ------------
<class 'pandas.core.frame.DataFrame'>
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   

                                                Name  Sex   Age  SibSp  Parch  \
0                            Braund, Mr. Owen Harris    0  22.0      1      0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...    0  38.0      1      0   

      Ticket     Fare Cabin Embarked  Deceased  
0  A/5 21171   7.2500     0        S         1  
1   PC 17599  71.2833   C85        C         0  


------------ Start Preprocessing ------------
     Sex   Age  Pclass  SibSp  Parch      Fare
0      0  22.0       3      1      0    7.2500
1      0  38.0       1      1      0   71.2833
2      0  26.0       3      0      0    7.9250
3      0  35.0       1      1      0   53.1000
4      0  35.0       3      0      0    8.0500
5      0   0.0       3      0      0    8.4583
6      0  54.0       1      0      0  

## 3. Divide dataset to (training set) and (test set)


Prevent from overfitting

In [14]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split


# split training data and test data (30% is test data)
X_train, X_test, y_train, y_test = train_test_split(
    dataset_X.as_matrix(), 
    dataset_Y.as_matrix(), 
    test_size = 0.2, 
    random_state = 42)



/Users/zhaohaonan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
/Users/zhaohaonan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


# Use sklearn

In [16]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(X_train, y_train)

/Users/zhaohaonan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/zhaohaonan/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [19]:
print(LR.score(X_test, y_test))

0.7374301675977654


# Output testset

In [48]:
# testdata preprocessing
testdata = pd.read_csv('data_set/test.csv')
dataset_t = testdata
testdata = testdata.fillna(0)
# convert ['male', 'female'] values to [1, 0]
testdata['Sex'] = testdata['Sex'].apply(lambda s: 1 if s == 'male' else 0)
# select features and labels for training
dataset_t = testdata[['Sex', 'Age', 'Pclass', 'SibSp', 'Parch', 'Fare']]

pid = testdata[['PassengerId']]

# list -> dataframe
pre = LR.predict(dataset_t)
pred = pd.DataFrame(pre, columns=['Survived'])
result = pd.concat([pid, pred], axis=1)

result.to_csv('titanic.csv', encoding = 'utf-8')